# Chapter 2: Stereo Visual Odometry - A Practical Demo

This notebook is the hands-on companion to the [Chapter 2 article](index.md). Here, we'll upgrade our visual odometry system from Chapter 1 by adding a second camera to solve the **scale problem**.

**Our goal**: To use two cameras (stereo vision) to calculate real-world depth and build a trajectory that has the correct scale in meters, without needing GPS or any external reference.

We will follow these key steps:
1.  **Setup and Data Loading**: Load stereo image pairs from both cameras.
2.  **Stereo Matching**: Match features between left and right images to find disparity.
3.  **Triangulation**: Calculate 3D positions of features using stereo geometry.
4.  **Temporal Tracking**: Track 3D points across time to estimate camera motion.
5.  **Pose Estimation with PnP**: Use 3D-to-2D correspondences to recover scaled motion.
6.  **Evaluation**: Compare our stereo VO result against Chapter 1's monocular VO and ground truth.

Let's get started!

In [ ]:
# --- Imports and Setup ---
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import cv2

# Add the project root directory to the Python path
# This allows us to import our custom modules from the `src` directory
# Assumes the notebook is in `chapters/2/`
project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))
print(f"Project root added to path: {project_root}")

# --- Our Project Modules ---
from src.datasets import KITTIDatasetFetcher
from src.feature_matching import FeatureMatcher
from src.pose_estimation import PoseEstimator
from src.visual_odometry import SimpleVisualOdometry, StereoVisualOdometry, run_vo_pipeline, run_stereo_vo_pipeline
from src.utils import load_image, plot_trajectory, compute_trajectory_error, plot_top_down_trajectory

# --- Matplotlib Setup ---
# Set a professional and readable style for our plots
plt.style.use('seaborn-v0_8-pastel')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['image.cmap'] = 'gray'
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("✅ Setup complete. All modules imported successfully.")

## Step 1: Download and Load the Stereo Dataset

We'll use the same KITTI Visual Odometry dataset from Chapter 1, but this time we'll load **both** the left and right camera images. The KITTI dataset was recorded with a stereo camera rig, where two cameras are mounted side-by-side with a known distance between them (the **baseline**).

This baseline distance is critical: it's what allows us to calculate real-world depth using triangulation.

In [ ]:
# --- Configuration ---
SEQUENCE_ID = "00"
DATA_DIR = project_root / "data"

# --- Download ---
print("Setting up dataset...")
# This will automatically download and unzip the data if it's not already present
kitti_fetcher = KITTIDatasetFetcher(data_dir=DATA_DIR)
kitti_fetcher.download_sequence(SEQUENCE_ID)
print(f"✅ Sequence '{SEQUENCE_ID}' is ready.")

# --- Load ---
print("\nLoading dataset components...")
# Load camera calibration parameters for BOTH cameras
calibration = kitti_fetcher.load_calibration(SEQUENCE_ID)
K_left = calibration['P0'][:, :3]   # Left camera intrinsics
K_right = calibration['P1'][:, :3]  # Right camera intrinsics

# Calculate the baseline from the projection matrices
# The baseline is encoded in the translation component of P1
baseline = abs(calibration['P1'][0, 3] / calibration['P1'][0, 0])

print(f"Left camera intrinsics (K_left):\n{K_left}")
print(f"\nRight camera intrinsics (K_right):\n{K_right}")
print(f"\nStereo baseline: {baseline:.3f} meters")

# Load the ground truth poses
ground_truth_poses = kitti_fetcher.load_poses(SEQUENCE_ID)

# Get image paths for BOTH cameras
left_image_paths = kitti_fetcher.get_image_paths(SEQUENCE_ID, camera="image_0")
right_image_paths = kitti_fetcher.get_image_paths(SEQUENCE_ID, camera="image_1")

print(f"\n✅ Loaded {len(left_image_paths)} stereo image pairs")
print(f"✅ Loaded {len(ground_truth_poses)} ground truth poses")

## Step 2: Visualize a Stereo Pair

Let's look at an example of a stereo image pair. Notice how the same scene is captured from two slightly different viewpoints. Objects closer to the camera will appear to "shift" more between the two views than distant objects. This shift is called **disparity**, and it's directly related to depth.

In [ ]:
# Load and display a stereo pair
frame_idx = 100
img_left = load_image(left_image_paths[frame_idx])
img_right = load_image(right_image_paths[frame_idx])

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].imshow(img_left, cmap='gray')
axes[0].set_title(f'Left Camera - Frame {frame_idx}', fontsize=14)
axes[0].axis('off')

axes[1].imshow(img_right, cmap='gray')
axes[1].set_title(f'Right Camera - Frame {frame_idx}', fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n👀 Notice: The two images show the same scene from slightly different angles.")
print("   Objects closer to the camera (like the road markings) shift more than distant objects (like the horizon).")

## Step 3: Stereo Matching and Triangulation

Now we'll demonstrate the core principle of stereo vision: matching features between the left and right images, then using the **disparity** to calculate the 3D position.

The key equation is:
$$
Z = \frac{\text{baseline} \times f}{\text{disparity}}
$$

Where:
- `Z` is the depth (distance from the camera)
- `f` is the focal length
- `disparity` is the horizontal pixel shift between the left and right images

Once we have `Z`, we can calculate the full 3D position `(X, Y, Z)` using the camera intrinsics.

In [ ]:
# Detect and match features in a stereo pair
matcher = FeatureMatcher(detector_type='ORB', max_features=2000)

# Detect features in both images
kp_left, desc_left = matcher.detect_features(img_left)
kp_right, desc_right = matcher.detect_features(img_right)

# Match features between left and right
stereo_matches = matcher.match_features(desc_left, desc_right)

print(f"Found {len(stereo_matches)} stereo matches")

# Visualize the matches
match_img = matcher.visualize_matches(
    img_left, kp_left,
    img_right, kp_right,
    stereo_matches,
    max_matches=50
)

plt.figure(figsize=(16, 8))
plt.imshow(match_img)
plt.title('Stereo Feature Matches (Left ↔ Right)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n✅ These matched features will be used to calculate 3D depth.")

## Step 4: Run Stereo Visual Odometry

Now we'll run our complete stereo VO pipeline on a sequence of frames. The system will:
1. For each frame pair, detect features in both left and right images
2. Match left-right to calculate 3D positions via triangulation
3. Track features between consecutive left images (time `t` → `t+1`)
4. Use PnP (Perspective-n-Point) to estimate camera motion from 3D-2D correspondences
5. Accumulate the motion to build the full trajectory

**Key difference from Chapter 1**: We now have **real-world scale** without needing to cheat with ground truth!

In [ ]:
# --- Run Stereo Visual Odometry ---
NUM_FRAMES = 200

print(f"Running stereo visual odometry on {NUM_FRAMES} frames...\n")

stereo_estimated_poses = run_stereo_vo_pipeline(
    camera_matrix_left=K_left,
    camera_matrix_right=K_right,
    baseline=baseline,
    left_image_paths=left_image_paths,
    right_image_paths=right_image_paths,
    ground_truth_poses=ground_truth_poses,
    num_frames=NUM_FRAMES,
    detector_type='ORB',
    max_features=2000
)

print(f"\n✅ Stereo VO complete! Estimated {len(stereo_estimated_poses)} poses.")

## Step 5: Compare with Monocular VO (Chapter 1)

To really appreciate what we've gained, let's run the monocular VO system from Chapter 1 on the same sequence and compare the results.

In [ ]:
# --- Run Monocular VO for comparison ---
print(f"Running monocular visual odometry (Chapter 1) for comparison...\n")

mono_estimated_poses = run_vo_pipeline(
    camera_matrix=K_left,
    image_paths=left_image_paths,
    ground_truth_poses=ground_truth_poses,
    num_frames=NUM_FRAMES,
    detector_type='ORB',
    max_features=2000
)

print(f"\n✅ Monocular VO complete! Estimated {len(mono_estimated_poses)} poses.")

## Step 6: Visualize and Compare Results

Now let's plot all three trajectories: ground truth, monocular VO, and stereo VO.

In [ ]:
# --- Top-Down Comparison ---
fig, ax = plt.subplots(1, 1, figsize=(14, 10))

# Ground truth
gt_pos = ground_truth_poses[:NUM_FRAMES, :3, 3]
ax.plot(gt_pos[:, 0], gt_pos[:, 2], 'k--', label='Ground Truth', linewidth=2, alpha=0.7)

# Monocular VO (from Chapter 1)
mono_pos = mono_estimated_poses[:, :3, 3]
ax.plot(mono_pos[:, 0], mono_pos[:, 2], 'r-', label='Monocular VO (Chapter 1)', linewidth=2, alpha=0.7)

# Stereo VO (Chapter 2)
stereo_pos = stereo_estimated_poses[:, :3, 3]
ax.plot(stereo_pos[:, 0], stereo_pos[:, 2], 'b-', label='Stereo VO (Chapter 2)', linewidth=2)

# Mark start and end
ax.scatter(gt_pos[0, 0], gt_pos[0, 2], c='green', s=150, label='Start', zorder=5, edgecolors='black')
ax.scatter(gt_pos[-1, 0], gt_pos[-1, 2], c='red', s=150, label='End', zorder=5, edgecolors='black')

ax.set_xlabel('X (m)', fontsize=12)
ax.set_ylabel('Z (m) - Forward', fontsize=12)
ax.set_title('Trajectory Comparison: Monocular vs Stereo Visual Odometry', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.axis('equal')

plt.tight_layout()
plt.show()

print("\n🎯 Key Observation:")
print("   The stereo VO trajectory (blue) has the CORRECT SCALE without any ground truth reference!")
print("   The monocular VO (red) had to be scaled using ground truth data to match reality.")

## Step 7: Quantitative Error Analysis

Let's calculate the error metrics for both systems to see how much stereo vision improved our accuracy.

In [ ]:
# --- Compute Errors ---
from src.utils import compute_trajectory_error

# Monocular VO error
mono_trans_errors, mono_rot_errors = compute_trajectory_error(
    mono_estimated_poses,
    ground_truth_poses[:len(mono_estimated_poses)]
)

# Stereo VO error
stereo_trans_errors, stereo_rot_errors = compute_trajectory_error(
    stereo_estimated_poses,
    ground_truth_poses[:len(stereo_estimated_poses)]
)

# --- Plot Error Comparison ---
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Translation error
axes[0].plot(mono_trans_errors, 'r-', label='Monocular VO', alpha=0.7, linewidth=2)
axes[0].plot(stereo_trans_errors, 'b-', label='Stereo VO', alpha=0.7, linewidth=2)
axes[0].set_ylabel('Translation Error (m)', fontsize=11)
axes[0].set_title('Position Error Over Time', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Rotation error
axes[1].plot(mono_rot_errors, 'r-', label='Monocular VO', alpha=0.7, linewidth=2)
axes[1].plot(stereo_rot_errors, 'b-', label='Stereo VO', alpha=0.7, linewidth=2)
axes[1].set_xlabel('Frame', fontsize=11)
axes[1].set_ylabel('Rotation Error (degrees)', fontsize=11)
axes[1].set_title('Rotation Error Over Time', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n📊 Error Summary:")
print("\nMonocular VO (Chapter 1):")
print(f"  Mean translation error: {np.mean(mono_trans_errors):.3f} m")
print(f"  RMS translation error:  {np.sqrt(np.mean(mono_trans_errors**2)):.3f} m")
print(f"  Mean rotation error:    {np.mean(mono_rot_errors):.3f}°")

print("\nStereo VO (Chapter 2):")
print(f"  Mean translation error: {np.mean(stereo_trans_errors):.3f} m")
print(f"  RMS translation error:  {np.sqrt(np.mean(stereo_trans_errors**2)):.3f} m")
print(f"  Mean rotation error:    {np.mean(stereo_rot_errors):.3f}°")

# Calculate improvement
improvement = (np.mean(mono_trans_errors) - np.mean(stereo_trans_errors)) / np.mean(mono_trans_errors) * 100
print(f"\n✨ Stereo VO reduced translation error by {improvement:.1f}%!")

## Step 8: 3D Trajectory Visualization

Finally, let's look at the 3D trajectories to see how both systems perform in recovering the altitude changes.

In [ ]:
# --- 3D Visualization ---
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Ground truth
ax.plot(gt_pos[:, 0], gt_pos[:, 2], -gt_pos[:, 1],
        'k--', label='Ground Truth', linewidth=2, alpha=0.7)

# Monocular VO
ax.plot(mono_pos[:, 0], mono_pos[:, 2], -mono_pos[:, 1],
        'r-', label='Monocular VO', linewidth=2, alpha=0.7)

# Stereo VO
ax.plot(stereo_pos[:, 0], stereo_pos[:, 2], -stereo_pos[:, 1],
        'b-', label='Stereo VO', linewidth=2)

# Start marker
ax.scatter(gt_pos[0, 0], gt_pos[0, 2], -gt_pos[0, 1],
           c='green', s=100, label='Start', zorder=5)

ax.set_xlabel('X (m)', fontsize=11)
ax.set_ylabel('Z (m) - Forward', fontsize=11)
ax.set_zlabel('-Y (m) - Altitude', fontsize=11)
ax.set_title('3D Trajectory Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.view_init(elev=20, azim=-60)

fig.tight_layout(pad=2.0)
plt.show()

print("\n🎉 Stereo VO provides scaled, 3D motion estimation without any external reference!")

## Conclusion

In this chapter, we successfully upgraded our visual odometry system by adding a second camera. The key achievements:

1. **Scale Recovery**: We can now estimate motion in **real-world meters** without any GPS or ground truth reference.
2. **Improved Accuracy**: Stereo VO typically provides more accurate trajectories than monocular VO.
3. **3D Reconstruction**: We can calculate the 3D structure of the scene, not just the camera path.

**Key Takeaway**: By leveraging the geometry of two cameras with a known baseline, we can transform 2D image observations into 3D world coordinates, solving the fundamental scale ambiguity problem of monocular vision.

**What's Next**: In future chapters, we'll explore SLAM (Simultaneous Localization and Mapping), loop closure detection, and sensor fusion with IMUs to build even more robust navigation systems.